# Workshop part 2 | Learn how to make a forecast
In this second part of the workshop, we will use the model trained in the first part and make a forecast with it. 

Note: if you were not able to train the model in the first part, we have trained one for you. It is in this folder: ``mlflow_trained_model``. It will follow in this tutorial how to use this. 

The learning points are:
- Hands on experience with using a trained model; 
- What data is required to make a forecast;
- Hands on experience using forecast pipeline;
- How the model gets automatically loaded;
- How the predictions compare to the measurements.

In [ ]:
# Import required packages.
import pandas as pd 
import numpy as np
import openstef
from openstef.pipeline.create_forecast import create_forecast_pipeline
from openstef.data_classes.prediction_job import PredictionJobDataClass

# Set plotly as the default pandas plotting backend.
pd.options.plotting.backend = 'plotly'

## Define the prediction job
The same as in workshop part 1, a prediction job has to be defined. As we are making a forecast for the model we trained in part 1, we can use the exact same prediction job. 

In [ ]:
# Define properties of training/prediction. We call this a 'prediction_job'. The same is used as in the first exercise.
pj = dict(id=287,
        model='xgb', 
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        forecast_type="demand", 
        lat=52.0,
        lon=5.0,
        horizon_minutes=2880,
        resolution_minutes=15,
        name="workshop_exercise_2",
        save_train_forecasts=True,
       )

pj=PredictionJobDataClass(**pj)

## Prepare the input data
Some other preparation of the input data is required for making a forecast. Namely, split into a test and train data set. 

Exercise: 
- Split the data into a train and test data set. Where the train dataset contains everything except the final 192 rows, and the test dataset contains this final 192 rows;

Hint: you can look at this [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html). 



In [ ]:
input_data=pd.read_csv("../data/input_data_sun_heavy.csv", index_col=0, parse_dates=True)

train_data= ...# Everything except the final 192 rows for training.
test_data= ...# Final 192 rows for testing.

In [ ]:
assert len(test_data)==192, "test data is of invalid length"

In [ ]:
# Prepare data to make the forecast.
realised=input_data.loc[test_data.index, 'load'].copy(deep=True)
to_forecast_data=input_data.copy(deep=True)
to_forecast_data.loc[test_data.index, 'load']= np.nan # Clear the load data for the part you want to forecast.

## Make the prediction
Now that the prediction job has been defined, a model has been trained and the input data is prepared, a forecast can be made. 

Exercise: 
- Using the prediction job, trained model and to_forecast_data, make a forecasting with the OpenSTEF pipeline;
- How long did it take to make a forecast?

Hint: look-up the correct pipeline on the OpenSTEF [website](https://openstef.github.io/openstef/user_guides.html).




In [ ]:
# Location where the model was stored in the last exercise.
mlflow_tracking_uri=r"./mlflow_trained_models" 

forecast=openstef.pipeline. ... (
    ...,
    ..., 
    mlflow_tracking_uri,
)

# Inspect the results
Now that the forecast has been made, the results can be analysed. 

Exercise: answer the following questions 
- Look at the results, when is the model accurate and when is it less accurate? Why?
- Look at the two weather features plotted, do you see correlation? 

In [ ]:
display(forecast.head())

In [ ]:
fig_forecast_realised=pd.concat([forecast["forecast"], realised], axis=1).plot()
fig_forecast_realised.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Load [MW]"
)
fig_forecast_realised.show()

In [ ]:
# Look at the normalized plots of both the radiation and forecast, do you recognize any paterns?

fig_forecast_radiation=pd.concat(
    [
        test_data["radiation"]/max(test_data["radiation"]),
        forecast["forecast"]/max(forecast["forecast"])
    ], axis=1).plot()
fig_forecast_radiation.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Normalized values"
)
display(fig_forecast_radiation.show())

In [ ]:
fig_forecast_windspeed=pd.concat(
    [
        test_data["windspeed"]/max(test_data["windspeed"]),
        forecast["forecast"]/max(forecast["forecast"])
    ], axis=1).plot()
fig_forecast_windspeed.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Normalized values"
)
display(fig_forecast_windspeed.show())

## Alter the input data 
In the code below, the radiation input data is divided by ten and thereafter a forecast is made with this new input data. Thus, with the same prediction job and trained model, a forecast is made using ten percent the 'sunshine' as input. 

Exercise: answer the following question: 
- What happens to the forecast when the radiation is divided by ten? Why? 


In [ ]:
# Divide the radiation data by ten.
to_forecast_data_rad=to_forecast_data.copy()
to_forecast_data_rad['radiation']=0.1*(to_forecast_data['radiation'])

# Make a forecast with this new input data.
mlflow_tracking_uri=r"./mlflow_trained_models" 

forecast_rad=create_forecast_pipeline(
    pj,
    to_forecast_data_rad, 
    mlflow_tracking_uri,
)

In [ ]:
# Inspect the results.
radiation_forecast_comparison = pd.DataFrame(
    test_data["radiation"]/max(test_data["radiation"])
)

radiation_forecast_comparison["forecast_full_radiation"] = forecast["forecast"]/max(forecast["forecast"])
radiation_forecast_comparison["forecast_little_radiation"] = forecast_rad["forecast"]/max(forecast_rad["forecast"])


fig_radiation_forecast_comparison=radiation_forecast_comparison.plot()

fig_radiation_forecast_comparison.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Normalized values"
)
display(fig_radiation_forecast_comparison.show())

## Bonus: Dashboard
Did you know that OpenSTEF has an eloborate dashboard which shows you everything you want to know about your forecast? Check it the dashboard documentation [here](https://raw.githack.com/OpenSTEF/.github/main/profile/html/openstef_dashboard_doc.html) . 

Which different in- and output components do you see in this dashboard? 
